# One-Shot Learning Example

The Jupyter Notebook should be launched in the folder **notebooks**.

In [1]:
import os
os.chdir('../src')
from osms.tts_modules.common.multispeaker import MultispeakerManager
import torch
import yaml
import warnings
warnings.filterwarnings("ignore")


Create a 5-second .wav file with someone speaking English and put it into the folder **audio_samples**.
Set the path to your .wav file in the attribute `SPEAKER_SPEECH_PATH` in `src/tts_modules/common/configs/main_config.yaml`.
We suggest to use the app [Audio Recorder](https://apps.apple.com/us/app/audio-recorder-wav-m4a/id1454488895) to record the voice. Set the sample rate to 16HGz there.

Create a .txt file with some sentences written in English and put it into the **texts** folder. Set the path to your .txt file in the attribute `INPUT_TEXTS_PATH` in `src/tts_modules/common/configs/main_config.yaml`.


The examples are already present in these folders.

In [2]:
with open(os.path.join(os.getcwd(), 'osms/tts_modules/common/configs/main_config.yaml'), "r") as ymlfile:
    main_config = yaml.load(ymlfile)
    
SPEAKER_SPEECH_PATH = "../audio_samples"
if not os.path.exists(SPEAKER_SPEECH_PATH):
    os.makedirs(SPEAKER_SPEECH_PATH)
    
INPUT_TEXTS_PATH = "../texts"
if not os.path.exists(INPUT_TEXTS_PATH):
    os.makedirs(INPUT_TEXTS_PATH)
    
OUTPUT_AUDIO_DIR = "../result_speech"
if not os.path.exists(OUTPUT_AUDIO_DIR):
    os.makedirs(OUTPUT_AUDIO_DIR)

In [3]:
multispeaker_manager = MultispeakerManager(main_config)
multispeaker_manager.inference()

Trainable Parameters in dVecModel: 1.424M
Loading DVecModel checkpoint from osms/tts_modules/encoder/models/checkpoints/encoder.pt
Trainable Parameters in Tacotron: 30.870M
Loading Tacotron checkpoint from osms/tts_modules/synthesizer/models/checkpoints/synthesizer.pt
Trainable Parameters in WaveRNN: 4.481M
Loading WaveRNN checkpoint from osms/tts_modules/vocoder/models/checkpoints/vocoder.pt


array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -3.12106918e-07, -8.36652316e-09, -0.00000000e+00])

The results will be available in the folder `result_speech`. The name of the file will be **result.wav**.

The usability will be further improved.